In [39]:
import MeCab

def get_iobcorpus(file_name):
    
    # アノテーション情報用意
    tag_number = []
    tag_vocab = {}
    with open(file_name, 'r') as f:
        for i in f.read().split("\n"):
            if len(i) != 0:
                t_s = int(i.split()[2])
                t_e = int(i.split()[3])
                tag_vocab[t_s] = i.split()[1]
                tag_number.append(t_s)
                tag_number.append(t_e)
    tag_number = sorted(tag_number)
    print(tag_number)
    
    # 原文データ用意
    # 改行の扱いを考える <=> データの統一
    mt = MeCab.Tagger('-Ochasen')
    result = mt.parseToNode("")
    data_text = ""
    file_name = file_name.replace("ann","txt")
    count = 0
    n = 0
    with open(file_name, 'r') as f:
        data_text = f.read()
        data_text = data_text.replace("\n",",") # mecabの改行コードが消える部分を補完
        result = mt.parseToNode(data_text)
        while result:
            count_list = []
            feature_list = result.feature.split(',')
            
            if feature_list[0] == 'BOS/EOS':
                result = result.next
                continue
                
            for i in range(len(str(result.surface))):                
                i = count + i
                count_list.append(i)
                if n < len(tag_number):
                    if tag_number[n] == i:
                        print(i)
                        n += 1
                        
            count += len(str(result.surface)) 
            print(count_list,result.surface,feature_list[:6])
            #print(result.surface,result.feature)
            result = result.next

    return data_text

In [40]:
get_iobcorpus("/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data1.ann")

[0, 4, 8, 27, 29, 42, 51, 55, 60, 124, 128, 138, 150, 162, 169, 245, 254, 258, 262, 292, 299, 374, 394, 398, 399, 401, 403, 407, 411, 526, 537, 541, 552, 554, 555, 602]
0
[0, 1, 2, 3] 琉球大学 ['名詞', '固有名詞', '組織', '*', '*', '*']
4
[4] で ['助詞', '格助詞', '一般', '*', '*', '*']
[5] は ['助詞', '係助詞', '*', '*', '*', '*']
[6] 、 ['記号', '読点', '*', '*', '*', '*']
[7] 「 ['記号', '括弧開', '*', '*', '*', '*']
8
[8, 9] 平成 ['名詞', '固有名詞', '一般', '*', '*', '*']
[10] ２ ['名詞', '数', '*', '*', '*', '*']
[11] ９ ['名詞', '数', '*', '*', '*', '*']
[12, 13] 年度 ['名詞', '接尾', '助数詞', '*', '*', '*']
[14] 第 ['接頭詞', '数接続', '*', '*', '*', '*']
[15] ４ ['名詞', '数', '*', '*', '*', '*']
[16] 回 ['名詞', '接尾', '助数詞', '*', '*', '*']
[17, 18] 学生 ['名詞', '一般', '*', '*', '*', '*']
[19] と ['助詞', '並立助詞', '*', '*', '*', '*']
[20, 21] 学長 ['名詞', '一般', '*', '*', '*', '*']
[22] と ['助詞', '格助詞', '一般', '*', '*', '*']
[23] の ['助詞', '連体化', '*', '*', '*', '*']
[24, 25] 懇談 ['名詞', 'サ変接続', '*', '*', '*', '*']
[26] 会 ['名詞', '接尾', '一般', '*', '*', '*']
27
[27] 」 ['記号

'琉球大学では、「平成２９年度第４回学生と学長との懇談会」を平成３０年１月２４日（水）に開催しました。,同懇談会の趣旨は、教育環境、教育方法の改善、学生生活支援、キャリア支援などについて学生と意見交換を行い、今後の教育改善、学生生活等の充実を図ることであり、平成１６年度から毎年開催しています。今回は、就職の内定を得た学部学生を対象に行い、学部学生代表１５名と大城学長、渡名喜理事・副学長、石川学長補佐、狩俣グローバル教育支援機構副機構長、遠藤キャリア教育センター長、嘉目監事及び尾原学生部長が出席しました。,同懇談会では、「就職支援としてのキャリア教育及びインターンシップ事業について」をテーマに、大学4年間の学修を通じた学生自身の就職に対する考え方の変化、就職活動における経験等について懇談するとともに、本学のキャリア教育や就職支援に対する要望等について、活発な意見交換を行いました。,同懇談会の最後に、大城学長から、「皆さんの目的意識が明確であるため、内定につながったと思う。また、学生としての質を保障するためのカリキュラムであるURGCCを通して能力を上げ、皆さんの人間としての幅も広がったと思うので、自信を持って、卒業しても自己を高めて欲しい。」とメッセージがあり、同懇談会を締めくくりました。,今後、学生から寄せられた意見等も踏まえ、本学の教育及び学生生活支援並びに就職活動支援の在り方について検討していくこととしています。,,,'

In [15]:
import os

file_list = []
# フォルダーからファイル検索
for file_name in os.listdir("/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt"):
    file_dir = "/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/{0}".format(file_name)
    if os.path.splitext(file_dir)[1] == ".ann":
        if os.path.getsize(file_dir) > 0:
            file_list.append(file_dir)

# 関数呼び出し
for f in file_list:
    print(f) 

/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data1.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data10.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data11.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data12.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data13.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data14.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data16.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data17.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data18.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/data19.ann
/Users/aharenchie/Desktop/ArticleAnalysis/brat-v1.3_Crunchy_Frog/data/R_data_txt/